In [2]:
import sys
import os
import psycopg2

# Add current directory to sys.path
sys.path.append(os.path.abspath("."))

from config_db import discover_databases


db_config = discover_databases()["employees"]
print(db_config)
conn = psycopg2.connect(
        host=db_config["host"],
        port=db_config["port"],
        database=db_config["dbname"],  #employees_db
        user=db_config["owner"],
        password=db_config["password"]
    )

{'dbname': 'employees', 'user': 'root', 'password': 'RM@123', 'host': 'localhost', 'port': 5432, 'owner': 'postgres'}


In [ ]:
conn = psycopg2.connect(
        host=db_config["host"],
        port=db_config["port"],
        database=db_config["dbname"],  #employees_db
        user=db_config["owner"],
        password=db_config["password"]
    )

cursor = conn.cursor()

# Drop the existing table if it exists
cursor.execute("DROP TABLE IF EXISTS employees;")

# Create table with updated schema (including degree_level)
create_table_query = """
CREATE TABLE employees (
    id           SERIAL PRIMARY KEY,
    first_name   TEXT NOT NULL,
    last_name    TEXT NOT NULL,
    email_id     TEXT UNIQUE NOT NULL,
    on_project   INTEGER NOT NULL CHECK (on_project IN (0,1)),
    tech_stack   TEXT[] NOT NULL,            -- ← array type
    exp          INTEGER NOT NULL,
    degree_level TEXT NOT NULL CHECK (degree_level IN ('Bachelor','Master','PhD'))
);
"""

# Execute and commit
cursor.execute(create_table_query)
conn.commit()

print("✅ 'employees' table created successfully.")

# Close connection
cursor.close()
conn.close()


✅ 'employees' table created successfully.


In [6]:
import psycopg2
import json
from config_db import discover_databases

# Connect to DB
db_config = discover_databases()["employees"]
conn = psycopg2.connect(
    host=db_config["host"],
    port=db_config["port"],
    database=db_config["dbname"],
    user=db_config["owner"],
    password=db_config["password"]
)
cursor = conn.cursor()

# Sample employee data
employees = [
    ("Alice", "Smith", "alice.smith@gmail.com", 1, ["Python", "FastAPI", "AWS"], 4, "Bachelor"),
    ("Bob", "Johnson", "bob.johnson@gmail.com", 0, ["Java", "Spring Boot", "PostgreSQL"], 6, "Master"),
    ("Charlie", "Brown", "charlie.brown@gmail.com", 1, ["LangChain", "OpenAI", "LangGraph"], 3, "PhD"),
    ("Diana", "Lee", "diana.lee@gmail.com", 0, ["Node.js", "Express", "MongoDB"], 2, "Bachelor"),
    ("Ethan", "White", "ethan.white@gmail.com", 1, ["React", "JavaScript", "Tailwind"], 5, "Bachelor"),
    ("Fiona", "Davis", "fiona.davis@gmail.com", 0, ["Python", "Airflow", "Spark"], 6, "Master"),
    ("George", "Harris", "george.harris@gmail.com", 1, ["LangGraph", "LangChain", "RAG"], 3, "PhD"),
    ("Hannah", "Moore", "hannah.moore@gmail.com", 0, ["Java", "AWS", "Kafka"], 4, "Bachelor"),
    ("Ian", "Clark", "ian.clark@gmail.com", 1, ["Next.js", "TypeScript", "Node.js"], 2, "Bachelor"),
    ("Jasmine", "Lewis", "jasmine.lewis@gmail.com", 0, ["GCP", "BigQuery", "Dataflow"], 7, "Master"),
    ("Kevin", "Walker", "kevin.walker@gmail.com", 1, ["LangChain", "GPT-4", "LLM"], 4, "PhD"),
    ("Laura", "Hall", "laura.hall@gmail.com", 0, ["FastAPI", "Docker", "PostgreSQL"], 3, "Bachelor"),
    ("Michael", "Young", "michael.young@gmail.com", 1, ["HuggingFace", "Transformers"], 5, "Master"),
    ("Nina", "Allen", "nina.allen@gmail.com", 0, ["TensorFlow", "Keras", "MLflow"], 6, "Master"),
    ("Oscar", "King", "oscar.king@gmail.com", 1, ["React", "Redux", "Node.js"], 4, "Bachelor"),
    ("Priya", "Patel", "priya.patel@gmail.com", 0, ["LangGraph", "Prompt Layer", "Pinecone"], 3, "PhD"),
    ("Quentin", "Scott", "quentin.scott@gmail.com", 1, ["Java", "MySQL", "Spring Boot"], 2, "Bachelor"),
    ("john", "Ryan", "john.ryan@gmail.com", 0, ["Python", "ETL", "Dask"], 5, "Master"),
    ("Sam", "Wright", "sam.wright@gmail.com", 1, ["Node.js", "GraphQL", "React"], 6, "Bachelor"),
    ("Tina", "Evans", "tina.evans@gmail.com", 0, ["GPT-4", "LangChain", "Vector DBs"], 4, "PhD"),
]

# Insert query
insert_query = """
    INSERT INTO employees 
    (first_name, last_name, email_id, on_project, tech_stack, exp, degree_level)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

# Execute insert
for emp in employees:
    tech_stack_json = json.dumps(emp[4])  # Convert list to JSON string
    cursor.execute(insert_query, (emp[0], emp[1], emp[2], emp[3], tech_stack_json, emp[5], emp[6]))

# Commit and close
conn.commit()
cursor.close()
conn.close()

print("✅ 20 sample employee records inserted.")


✅ 20 sample employee records inserted.


In [ ]:
import psycopg2
from psycopg2.extras import execute_values
from config_db import discover_databases

# --------------------------------------------------------------------------
# 1 · Connect
# --------------------------------------------------------------------------
db = discover_databases()["employees"]
conn = psycopg2.connect(
    host=db["host"],
    port=db["port"],
    dbname=db["dbname"],
    user=db["owner"],
    password=db["password"],
)
cur = conn.cursor()

# --------------------------------------------------------------------------
# 2 · Sample data  (unchanged)
# --------------------------------------------------------------------------
employees = employees = [
    ("Alice", "Smith", "alice.smith@gmail.com", 1, ["Python", "FastAPI", "AWS"], 4, "Bachelor"),
    ("Bob", "Johnson", "bob.johnson@gmail.com", 0, ["Java", "Spring Boot", "PostgreSQL"], 6, "Master"),
    ("Charlie", "Brown", "charlie.brown@gmail.com", 1, ["LangChain", "OpenAI", "LangGraph"], 3, "PhD"),
    ("Diana", "Lee", "diana.lee@gmail.com", 0, ["Node.js", "Express", "MongoDB"], 2, "Bachelor"),
    ("Ethan", "White", "ethan.white@gmail.com", 1, ["React", "JavaScript", "Tailwind"], 5, "Bachelor"),
    ("Fiona", "Davis", "fiona.davis@gmail.com", 0, ["Python", "Airflow", "Spark"], 6, "Master"),
    ("George", "Harris", "george.harris@gmail.com", 1, ["LangGraph", "LangChain", "RAG"], 3, "PhD"),
    ("Hannah", "Moore", "hannah.moore@gmail.com", 0, ["Java", "AWS", "Kafka"], 4, "Bachelor"),
    ("Ian", "Clark", "ian.clark@gmail.com", 1, ["Next.js", "TypeScript", "Node.js"], 2, "Bachelor"),
    ("Jasmine", "Lewis", "jasmine.lewis@gmail.com", 0, ["GCP", "BigQuery", "Dataflow"], 7, "Master"),
    ("Kevin", "Walker", "kevin.walker@gmail.com", 1, ["LangChain", "GPT-4", "LLM"], 4, "PhD"),
    ("Laura", "Hall", "laura.hall@gmail.com", 0, ["FastAPI", "Docker", "PostgreSQL"], 3, "Bachelor"),
    ("Michael", "Young", "michael.young@gmail.com", 1, ["HuggingFace", "Transformers"], 5, "Master"),
    ("Nina", "Allen", "nina.allen@gmail.com", 0, ["TensorFlow", "Keras", "MLflow"], 6, "Master"),
    ("Oscar", "King", "oscar.king@gmail.com", 1, ["React", "Redux", "Node.js"], 4, "Bachelor"),
    ("Priya", "Patel", "priya.patel@gmail.com", 0, ["LangGraph", "Prompt Layer", "Pinecone"], 3, "PhD"),
    ("Quentin", "Scott", "quentin.scott@gmail.com", 1, ["Java", "MySQL", "Spring Boot"], 2, "Bachelor"),
    ("john", "Ryan", "john.ryan@gmail.com", 0, ["Python", "ETL", "Dask"], 5, "Master"),
    ("Sam", "Wright", "sam.wright@gmail.com", 1, ["Node.js", "GraphQL", "React"], 6, "Bachelor"),
    ("Tina", "Evans", "tina.evans@gmail.com", 0, ["GPT-4", "LangChain", "Vector DBs"], 4, "PhD"),
]

# --------------------------------------------------------------------------
# 3 · Bulk insert  (lists → text[])
# --------------------------------------------------------------------------
insert_q = """
    INSERT INTO employees
      (first_name, last_name, email_id, on_project, tech_stack, exp, degree_level)
    VALUES %s
"""

# build a list of tuples in exactly the same order as the columns
records = [
    (first, last, email, on_proj, stack, years, degree)
    for first, last, email, on_proj, stack, years, degree in employees
]

execute_values(cur, insert_q, records)   # psycopg2 adapts list → array
conn.commit()

cur.close()
conn.close()
print("✅  20 sample employee records inserted into TEXT[] column.")
